In [ ]:
from crpropa import *

In [34]:
from pylab import *

# Synthetic turbulent magnetic field with CRPropa

## 1. Creation of field

### Properties of grid and field

In [ ]:
# Grid
spacing = 10 * pc
grid_number = 100

# Field
Brms = 5 * nG
lmin = 50 * pc 
lmax = 1000 * pc 
index = 5./3. 
random_seed = 42

# Creation of turbulent field
spectrum = SimpleTurbulenceSpectrum(Brms, lmin, lmax, index)
grid = GridProperties(Vector3d(0), grid_number, spacing)
BField = SimpleGridTurbulence(spectrum, grid, random_seed)

# Save field
dumpGrid(BField.getGrid(), 'kolmogorov_field.dat')

### Check of field properties

In [ ]:
print('Lc = {:.1f} kpc'.format(BField.getCorrelationLength() / pc))  
print('sqrt(<B^2>) = {:.1f} nG'.format(BField.getBrms() / nG))   
print('<|B|> = {:.1f} nG'.format(BField.getMeanFieldStrength() / nG))

## 2. Diagnostic of field

### Extraction of field components at random positions

In [ ]:
# Storing list
N = int(1e6)
Bx = [0] * N
By = [0] * N
Bz = [0] * N

# Evaluation of field
for i in range(0,N):
    x = random.randrange(0,10000)
    y = random.randrange(0,10000)
    z = random.randrange(0,10000)
    field = BField.getField(Vector3d(x,y,z) * pc) / nG
    Bx[i] = field[0]
    By[i] = field[1]
    Bz[i] = field[2]

# Transforming to arrays
Bx = np.array(Bx)
By = np.array(By)
Bz = np.array(Bz)
B = np.sqrt(Bx**2 + By*+2 + Bz**2)
Bx_sqr = Bx**2
By_sqr = By**2
Bz_sqr = Bz**2

### Check of mean values

In [ ]:
Bx_mean = np.mean(Bx)
By_mean = np.mean(By)
Bz_mean = np.mean(Bz)

print('Mean Bx', Bx_mean)
print('Mean By', By_mean)
print('Mean Bz', Bz_mean)

### Plotting distributions

In [ ]:
# Plot of field components
plt.figure(figsize = (15, 10))
#
plt.subplot(2, 2, 1)
counts, bins = np.histogram(Bx, bins=50)
plt.hist(bins[:-1], bins, weights=counts)
plt.xlabel(r'$\delta B_x$ [nG]', fontsize=17)
plt.grid()
#
plt.subplot(2, 2, 2)
counts, bins = np.histogram(By, bins=50)
plt.hist(bins[:-1], bins, weights=counts)
plt.xlabel(r'$\delta B_y$ [nG]', fontsize=17)
plt.grid()
#
plt.subplot(2, 2, 3)
counts, bins = np.histogram(Bz, bins=50)
plt.hist(bins[:-1], bins, weights=counts)
plt.xlabel(r'$\delta B_z$ [nG]', fontsize=17)
plt.grid()
#
plt.savefig('mean_field_components_kolmogorov.pdf')

In [ ]:
# Plot of squared field components
plt.figure(figsize = (15, 10))
#
plt.subplot(2, 2, 1)
counts, bins = np.histogram(Bx_sqr, bins=50)
plt.hist(bins[:-1], bins, weights=counts)
plt.xlabel(r'$(\delta B_x)^2$', fontsize=17)
plt.yscale('log')
plt.grid()
#
plt.subplot(2, 2, 2)
counts, bins = np.histogram(By_sqr, bins=50)
plt.hist(bins[:-1], bins, weights=counts)
plt.xlabel(r'$(\delta B_y)^2$', fontsize=17)
plt.yscale('log')
plt.grid()
#
plt.subplot(2, 2, 3)
counts, bins = np.histogram(Bz_sqr, bins=50)
plt.hist(bins[:-1], bins, weights=counts)
plt.xlabel(r'$(\delta B_z)^2$', fontsize=17)
plt.yscale('log')
plt.grid()
#
plt.savefig('squared_field_components_kolmogorov.pdf')